In [ ]:
!pip install boto3
!pip install pandas
!pip install numpy
!pip install sqlalchemy
!pip install psycopg2
!pip install pymysql

In [ ]:
import boto3
import pandas as pd
import numpy as np
import pymysql
import psycopg2
import configparser
from sqlalchemy import create_engine


In [ ]:
config = configparser.ConfigParser()
try:
    config.read('../config.conf')
    if not config.sections():
        print("No se encontraron secciones en el archivo de configuración.")
    else:
        print("Archivo de configuración cargado correctamente.")
except Exception as e:
    print("Ocurrió un error al leer el archivo de configuración:", e)
config

In [ ]:
aws_rds_conn = boto3.client('rds',
                             aws_access_key_id=config.get('IAM', 'ACCESS_KEY'),
                             aws_secret_access_key=config.get('IAM', 'SECRET_ACCESS_KEY'),
                             region_name=config.get('IAM', 'REGION'))



In [ ]:
aws_ec2_conn = boto3.client('ec2',
                             aws_access_key_id=config.get('IAM', 'ACCESS_KEY'),
                             aws_secret_access_key=config.get('IAM', 'SECRET_ACCESS_KEY'),
                             region_name=config.get('IAM', 'REGION'))


In [ ]:
try:
    response = aws_rds_conn.create_db_instance(
        DBInstanceIdentifier=config.get('TRANSACC', 'DB_INSTANCE_ID'),
        DBName=config.get('TRANSACC', 'DB_NAME'),
        DBInstanceClass='db.t3.micro',
        Engine='mysql',
        MasterUsername=config.get('TRANSACC', 'DB_USER'),
        MasterUserPassword=config.get('TRANSACC', 'DB_PASSWORD'),
        Port=int(config.get('TRANSACC', 'DB_PORT')),
        PubliclyAccessible=True,
        VpcSecurityGroupIds=[config.get('VPC', 'SECURITY_GROUP')],
        AllocatedStorage=20
    )
    print(response)
except aws_rds_conn.exceptions.DBInstanceAlreadyExistsFault as ex:
    print("La instancia de base de datos MySQL ya existe.")
except Exception as ex:
    print("Error al crear la base de datos MySQL:", ex)


In [ ]:
# Crear la base de datos PostgreSQL para DWH
try:
    response = aws_rds_conn.create_db_instance(
        DBInstanceIdentifier=config.get('DWH', 'DB_INSTANCE_ID'),
        DBName=config.get('DWH', 'DB_NAME'),
        DBInstanceClass='db.t3.micro',
        Engine='postgres',
        MasterUsername=config.get('DWH', 'DB_USER'),
        MasterUserPassword=config.get('DWH', 'DB_PASSWORD'),
        Port=int(config.get('DWH', 'DB_PORT')),
        PubliclyAccessible=True,
        VpcSecurityGroupIds=[config.get('VPC', 'SECURITY_GROUP')],
        AllocatedStorage=20
    )
    print(response)
except aws_rds_conn.exceptions.DBInstanceAlreadyExistsFault as ex:
    print("La instancia de base de datos PostgreSQL ya existe.")
except Exception as ex:
    print("Error al crear la base de datos PostgreSQL:", ex)


In [ ]:
def generate_key_pair(key_name):    
    ec2_client = boto3.client('ec2',
                              aws_access_key_id=config.get('IAM', 'ACCESS_KEY'),
                              aws_secret_access_key=config.get('IAM', 'SECRET_ACCESS_KEY'),
                              region_name=config.get('IAM', 'REGION'))
    
    try:
        key_pair = ec2_client.create_key_pair(KeyName=key_name)
        private_key = key_pair['KeyMaterial']
        
        # Guardar la clave privada en un archivo
        with open(f'{key_name}.pem', 'w') as file:
            file.write(private_key)
        
        print(f'Key pair "{key_name}" creado exitosamente.')
        print(f'Clave privada guardada en: {key_name}.pem')
    
    except ec2_client.exceptions.ClientError as e:
        if 'InvalidKeyPair.Duplicate' in str(e):
            print(f'El key pair "{key_name}" ya existe.')
        else:
            print('Error al crear el key pair:', e)


# Nombre del key pair a generar
key_name = 'key-pair-project'

# Generar el key pair
generate_key_pair(key_name)

In [ ]:
print(key_name)

In [ ]:
try:
    response = aws_ec2_conn.run_instances(
        ImageId=config.get('EC2', 'IMAGE_ID'),
        InstanceType='t2.micro',
        KeyName=key_name,
        MinCount=1,
        MaxCount=1,
        SecurityGroupIds=[config.get('VPC', 'SECURITY_GROUP')]
    )
    instance_id = response['Instances'][0]['InstanceId']
    print("Instancia EC2 creada con ID:", instance_id)
except Exception as ex:
    print("Error al crear la instancia EC2:", ex)


In [ ]:
try:
    instance = aws_rds_conn.describe_db_instances(DBInstanceIdentifier=config.get('TRANSACC', 'DB_INSTANCE_ID'))
    mysql_hostname = instance.get('DBInstances')[0].get('Endpoint').get('Address')
    print("Hostname de la base de datos MySQL:", mysql_hostname)
except Exception as ex:
    print("Error al obtener el hostname de la base de datos MySQL:", ex)


In [ ]:
try:
    instance = aws_rds_conn.describe_db_instances(DBInstanceIdentifier=config.get('DWH', 'DB_INSTANCE_ID'))
    postgresql_hostname = instance.get('DBInstances')[0].get('Endpoint').get('Address')
    print("Hostname de la base de datos PostgreSQL:", postgresql_hostname)
except Exception as ex:
    print("Error al obtener el hostname de la base de datos PostgreSQL:", ex)
    

### Agregar informacion a archivo JSON para usarlo a traves de los Notebooks

In [ ]:
import json

# Suponiendo que config es un diccionario y mysql_hostname es una cadena
data_to_share = {
    'mysql_hostname': mysql_hostname,
    'postgresql_hostname': postgresql_hostname,
    'EC2_instance_id': instance_id,
    'KEY_PAIR': key_name
}

with open('../shared_data.json', 'w') as outfile:
    json.dump(data_to_share, outfile)

In [ ]:
# Eliminar la base de datos MySQL de transacciones
try:
    response = aws_rds_conn.delete_db_instance(
        DBInstanceIdentifier=config.get('TRANSACC', 'DB_INSTANCE_ID'),
        SkipFinalSnapshot=True
    )
    print(response)
except Exception as ex:
    print("Error al eliminar la base de datos MySQL:", ex)


In [ ]:
# Eliminar la base de datos PostgreSQL para DWH
try:
    response = aws_rds_conn.delete_db_instance(
        DBInstanceIdentifier=config.get('DWH', 'DB_INSTANCE_ID'),
        SkipFinalSnapshot=True
    )
    print(response)
except Exception as ex:
    print("Error al eliminar la base de datos PostgreSQL:", ex)



In [ ]:
# Eliminar la instancia EC2
try:
    response = aws_ec2_conn.terminate_instances(
        InstanceIds=[instance_id]
    )
    print(response)
except Exception as ex:
    print("Error al eliminar la instancia EC2:", ex)

In [ ]:
try:
    response = aws_ec2_conn.delete_key_pair(
        KeyName=config.get('EC2', 'KEY_PAIR')
    )
    print(response)
except Exception as ex:
    print("Error al eliminar el par de claves:", ex)